In [45]:
import os 
from dotenv import load_dotenv
load_dotenv('../../.env')

model=os.environ.get('model')
print(model)

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get('AZURE_API_BASE')
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get('AZURE_API_KEY')
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ.get('AZURE_API_VERSION')

azure/gpt-4


In [46]:
from crewai import Agent, Task, Crew
from crewai.memory.short_term.short_term_memory import ShortTermMemory 
from crewai.memory.memory import Memory
from langchain.vectorstores import FAISS
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI

## Example utility function that simulates keyword-based classification.
In a production CrewAI implementation, this might be replaced with calls to Azure OpenAI for natural language classification

In [47]:
def classify_query(text, keywords): 
    text_lower = text.lower() 
    for key in keywords: 
        if key in text_lower: 
            return True 
    return False

## Base Agent class for all agents

In [48]:
class Agent: 
    def __init__(self, name): 
        self.name = name
    
    def process(self, query):
        raise NotImplementedError("Agent process method must be implemented by subclasses")

## Specialized Agents for Travel

In [49]:
#########################
## Travel related agents
#########################

class FlightAgent(Agent): 
    def process(self, query): 
        return f"{self.name} handling flight query: {query}"

class TrainAgent(Agent): 
    def process(self, query): 
        return f"{self.name} handling train query: {query}"

class TravelAssistantAgent(Agent): 
    def process(self, query): 
        return f"{self.name} handling general travel query: {query}"

#########################
## Finance related agents
#########################
class StockAgent(Agent): 
    def process(self, query): 
        return f"{self.name} handling stock query: {query}"

class MutualFundAgent(Agent): 
    def process(self, query): 
        return f"{self.name} handling mutual fund query: {query}"

class FinanceAssistantAgent(Agent): 
    def process(self, query): 
        return f"{self.name} handling general finance query: {query}"        

## Supervisor Agents that route queries in their respective domains

In [53]:
class TravelSupervisor(Agent): 
    def __init__(self, name): 
        super().__init__(name) 
        self.flight_agent = FlightAgent("flight_agent") 
        self.train_agent = TrainAgent("train_agent") 
        self.assistant_agent = TravelAssistantAgent("travel_assistant_agent")

    def process(self, query):
        # Determine which travel agent the query should be sent to
        if classify_query(query, ["flight", "airline", "airplane"]):
            return self.flight_agent.process(query)
        elif classify_query(query, ["train", "rail"]):
            return self.train_agent.process(query)
        else:
            return self.assistant_agent.process(query)

class FinanceSupervisor(Agent): 
    def __init__(self, name): 
        super().__init__(name) 
        self.stock_agent = StockAgent("stock_agent") 
        self.mutual_fund_agent = MutualFundAgent("mutual_fund_agent") 
        self.assistant_agent = FinanceAssistantAgent("finance_assistant_agent")

    def process(self, query):
        # Determine which finance agent the query should be sent to
        if classify_query(query, ["stock", "share", "market"]):
            return self.stock_agent.process(query)
        elif classify_query(query, ["mutual fund", "investment fund"]):
            return self.mutual_fund_agent.process(query)
        else:
            return self.assistant_agent.process(query)        

## Master Supervisor which routes to either travel or finance supervisor

In [54]:
class MasterSupervisor(Agent): 
    def __init__(self, name): 
        super().__init__(name) 
        self.travel_supervisor = TravelSupervisor("travel_supervisor") 
        self.finance_supervisor = FinanceSupervisor("finance_supervisor")

    def process(self, query):
        # Use simple keyword matching to decide on travel vs finance.
        # In a real-world CrewAI implementation, you would leverage Azure OpenAI's NL understanding.
        if classify_query(query, ["flight", "train", "travel", "airline", "rail"]):
            response = self.travel_supervisor.process(query)
            return f"Routed to travel domain: {response}"
        elif classify_query(query, ["stock", "mutual fund", "finance", "investment"]):
            response = self.finance_supervisor.process(query)
            return f"Routed to finance domain: {response}"
        else:
            return "Unable to determine query domain, please rephrase."

## Simulate processing customer queries.

In [55]:
master = MasterSupervisor("master_supervisor")

queries = [
    "I need to book a flight ticket for my vacation.",
    "What's the current stock price for ABC Corp?",
    "Can you help me with train timings for tomorrow?",
    "I want to know more about mutual fund investments.",
    "Help me plan my travel itinerary."
]

for query in queries:
    print("Query:", query)
    result = master.process(query)
    print("Response:", result)
    print("-" * 50)

Query: I need to book a flight ticket for my vacation.
Response: Routed to travel domain: flight_agent handling flight query: I need to book a flight ticket for my vacation.
--------------------------------------------------
Query: What's the current stock price for ABC Corp?
Response: Routed to finance domain: stock_agent handling stock query: What's the current stock price for ABC Corp?
--------------------------------------------------
Query: Can you help me with train timings for tomorrow?
Response: Routed to travel domain: train_agent handling train query: Can you help me with train timings for tomorrow?
--------------------------------------------------
Query: I want to know more about mutual fund investments.
Response: Routed to finance domain: mutual_fund_agent handling mutual fund query: I want to know more about mutual fund investments.
--------------------------------------------------
Query: Help me plan my travel itinerary.
Response: Routed to travel domain: travel_assista